In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import tabulate
from collections import  namedtuple
from statsmodels.stats.power import TTestIndPower, TTestPower, zt_ind_solve_power

### Odkazy na relevantní materiály
- [Statistické testy ve scipy](https://docs.scipy.org/doc/scipy/reference/stats.html#statistical-tests)
- [Statsmodels dokumentace](https://www.statsmodels.org/dev/index.html)
- [Skripta](https://dostal.vyzkum-psychologie.cz/skripta_statistika.pdf)
- [Zadání příkladů](https://dostal.vyzkum-psychologie.cz/stat/index.php)

# Úkol 11: Analýza síly testu
Plánujeme realizaci dvou výzkumných designů.

První z nich je experiment a budeme v něm pomocí t-testu pro dva nezávislé výběry srovnávat kontrolní a experimentální skupinu. Jelikož se jedná o replikaci dříve provedené studie, máme k dispozici přesnou hodnotu očekávané míry účinku delta.

Druhý výzkum je dotazníkové šetření, kde chceme zmapovat korelaci mezi výsledky ve dvou psychologických testech. Z literatury známe opět poměrně přesně očekávanou velikost Pearsonova korelačního koeficientu.

Očekávané hodnoty měr účinku delta a ró jsou uvedeny níže. Při analýze síly testu korelačního koeficientu použijte Z transformaci (bez jakýchkoli korekcí). Hypotézy budeme testovat jako oboustranné na hladině významnosti alfa = 0.05.


---

Načtení a vizualizace dat

In [2]:
df = pd.read_csv("data.csv", index_col=0, delimiter=";")
df

,Delta,Ro
#,,
40,"1,39","0,63"


In [3]:
alpha = 0.05
delta = float(df.iloc[0]["Delta"].replace(",", "."))
rho = float(df.iloc[0]["Ro"].replace(",", "."))

print(f"{alpha=}, {delta=}, {rho=}")

alpha=0.05, delta=1.39, rho=0.63


---

Jakou sílu bude mít t-test pro dva výběry z první studie, pokud máme k dispozici dvě skupiny, každou o rozsahu 11 jedinců?

In [4]:
n1 = 11
n2 = 11
ratio = n2/n1

test = TTestIndPower()
test.power(effect_size=delta, alpha=alpha, nobs1=n1, ratio=ratio)

0.8728305913865801

[(Skripta, str. 127)](https://dostal.vyzkum-psychologie.cz/skripta_statistika.pdf#page=127)

Jaký rozsah skupin budeme potřebovat, když chceme v první studii dosáhnout síly testu nejméně 0.9. Předpokládejme stejně velké skupiny, uveďte rozsah jedné z nich.

In [5]:
desired_power = 0.9

test = TTestIndPower()
nobs1 = test.solve_power(effect_size=delta, alpha=alpha, nobs1=None, ratio=ratio, power=desired_power)
nobs1_ceil = int(np.ceil(nobs1))
print(f"{nobs1=}, {nobs1_ceil=}")


nobs1=11.92444142277259, nobs1_ceil=12


Jak velký soubor budeme potřebovat, když design první studie změníme tak, že místo kontrolní skupiny použijeme hodnotu z dřívějšího rozsáhlého šetření. Míra účinku zůstává stejná.

In [6]:
desired_power = 0.9

test = TTestPower()
nobs = test.solve_power(effect_size=delta, alpha=alpha, nobs=None, power=desired_power)
nobs_ceil = int(np.ceil(nobs))
print(f"{nobs=}, {nobs_ceil=}")

nobs=7.6136427640618916, nobs_ceil=8


Jakou sílu bude mít test korelačního koeficientu z druhé studie, když máme k dispozici 60 párových měření?

In [7]:
nobs = 60

def fisher_z_transform(r, n):
    return 1 / 2 * np.log((1+r) / (1-r)) * np.sqrt(n - 3)

def z_power(rho, alpha, n):
    u = fisher_z_transform(rho, n)
    theta_1 = stats.norm.ppf(1-(alpha/2))
    theta_2 = stats.norm.ppf(alpha/2)
    power = 1 - stats.norm.cdf(theta_1 - u) + stats.norm.cdf(theta_2 - u)
    return power

power = z_power(rho, alpha, nobs)
print(f"{power=}")

power=0.9998624075702673


[(Skripta, str. 133)](https://dostal.vyzkum-psychologie.cz/skripta_statistika.pdf#page=133)

Jak velký výzkumný soubor budeme potřebovat, když chceme v druhé studii dosáhnout síly testu nejméně 0.9.

In [8]:
n = np.arange(3, 30)

power = z_power(rho, alpha, n)
list(zip(n, power))

[(3, 0.05),
 (4, 0.11496054655952234),
 (5, 0.1823437153174399),
 (6, 0.25017490480439053),
 (7, 0.31692203910859956),
 (8, 0.3814337620700983),
 (9, 0.4428801450866833),
 (10, 0.5006977541920752),
 (11, 0.5545401034807789),
 (12, 0.6042339361668162),
 (13, 0.6497413680851258),
 (14, 0.6911276580991801),
 (15, 0.7285342012158654),
 (16, 0.7621562460578178),
 (17, 0.7922247975300507),
 (18, 0.8189921617503759),
 (19, 0.8427206112315568),
 (20, 0.8636736846773463),
 (21, 0.8821096808195004),
 (22, 0.8982769546320045),
 (23, 0.9124106736405626),
 (24, 0.9247307396469191),
 (25, 0.9354406255911316),
 (26, 0.9447269176754294),
 (27, 0.95275938890495),
 (28, 0.959691461803988),
 (29, 0.9656609453872083)]

23